In [1]:
#hide
#skip
%config Completer.use_jedi = False
# upgrade fastrl on colab
! [ -e /content ] && pip install -Uqq fastrl['dev'] pyvirtualdisplay && \
                     apt-get install -y xvfb python-opengl > /dev/null 2>&1 
# NOTE: IF YOU SEE VERSION ERRORS, IT IS SAFE TO IGNORE THEM. COLAB IS BEHIND IN SOME OF THE PACKAGE VERSIONS

In [2]:
# hide
from fastcore.imports import in_colab
# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.showdoc import *
    from nbdev.imports import *
    if not os.environ.get("IN_TEST", None):
        assert IN_NOTEBOOK
        assert not IN_COLAB
        assert IN_IPYTHON
else:
    # Virutual display is needed for colab
    from pyvirtualdisplay import Display
    display = Display(visible=0, size=(400, 300))
    display.start()

In [3]:
# default_exp agents.dqn.targets

In [4]:
# export
# Python native modules
import os
from collections import deque
from typing import *
# Third party libs
import torch
from torch.nn import *
from fastcore.all import *
from fastai.learner import *
from fastai.torch_basics import *
from fastai.torch_core import *
from fastai.callback.all import *
# Local modules
from fastrl.data.block_simple import *
from fastrl.data.gym import *
from fastrl.agent import *
from fastrl.core import *
from fastrl.agents.dqn.core import *

# DQN Targets + N-Step
> A Bare-Bones DQN is usually extremely unstable. Target models can eleviate this. We also support First-Last N steps better.

In [15]:
# export
class DQNTargetTrainer(Callback):
    
    def __init__(self,n_batch=0,target_sync=300,discount=0.99,n_steps=1):
        store_attr()
        self._xb=None  
        
    def before_fit(self):
        self.learn.target_model=deepcopy(self.learn.model.model)
        self.n_batch=0
    
    def after_pred(self):
        self.learn.yb=self.xb
        self.learn.xb=self.xb[0]
        self._xb=({k:v.clone() for k,v in self.xb.items()},)
        self.learn.done_mask=self.xb['done'].reshape(-1,)
        self.learn.next_q=self.target_model(self.xb['next_state']).max(dim=1).values.reshape(-1,1)
        self.learn.next_q[self.done_mask]=0
        self.learn.targets=self.xb['reward']+self.learn.next_q*(self.discount**self.n_steps)
        self.learn.pred=self.learn.model.model(self.xb['state'])
        t_q=self.pred.clone()
        t_q.scatter_(1,self.xb['action'],self.targets)
        self.learn.yb=(t_q,)
        
    def before_backward(self): self.learn.xb=self._xb
        
    def after_batch(self):
        if self.n_batch%self.target_sync==0:
            self.target_model.load_state_dict(self.learn.model.state_dict())
        self.n_batch+=1

In [22]:
ExperienceReplay??

Init signature: ExperienceReplay(*args, bs=16, max_sz=200, warmup_sz=100, **kwargs)
Docstring:      Basic class handling tweaks of the training loop by changing a `Learner` in various events
Source:        
class ExperienceReplay(Callback):
    def __init__(self,*args,bs=16,max_sz=200,warmup_sz=100,**kwargs):
        store_attr()
        self.memory=None
        self.pointer=0

    def after_pred(self):
        xb=BD(self.learn.xb[0]).mapv(to_detach)
        if self.memory is None:            self.memory=xb
        elif self.memory.bs()<self.max_sz: self.memory+=xb
        else:
            self.memory=self.memory[:self.pointer]+xb+self.memory[self.pointer+xb.bs():]
            self.pointer+=xb.bs()
            if self.pointer>self.max_sz: self.pointer=0
        with torch.no_grad():
            idxs=np.random.randint(0,self.memory.bs(),self.bs).tolist()
            self.learn.xb=(self.memory[idxs].mapv(to_device),)

        if self.memory.bs()<self.warmup_sz: raise CancelBatchExceptio

In [28]:
BD(learn.cbs[3].memory)[:100].pandas(jupyter_nrows=300)

,state,next_state,done,all_done,env_id,worker_id,action,episode_id,accum_rewards,reward,step
0,"torch.Size([100, 4])","torch.Size([100, 4])",False,False,0,0,0,1,1.0,2.9701,1
1,"torch.Size([100, 4])","torch.Size([100, 4])",False,False,0,0,0,1,2.0,2.9701,2
2,"torch.Size([100, 4])","torch.Size([100, 4])",False,False,0,0,1,1,3.0,2.9701,3
3,"torch.Size([100, 4])","torch.Size([100, 4])",False,False,0,0,0,1,4.0,2.9701,4
4,"torch.Size([100, 4])","torch.Size([100, 4])",False,False,0,0,0,1,5.0,2.9701,5
5,"torch.Size([100, 4])","torch.Size([100, 4])",False,False,0,0,0,1,6.0,2.9701,6
6,"torch.Size([100, 4])","torch.Size([100, 4])",False,False,0,0,1,1,7.0,2.9701,7
7,"torch.Size([100, 4])","torch.Size([100, 4])",False,False,0,0,0,1,8.0,2.9701,8
8,"torch.Size([100, 4])","torch.Size([100, 4])",False,False,0,0,1,1,9.0,2.9701,9
9,"torch.Size([100, 4])","torch.Size([100, 4])",True,False,0,0,0,1,10.0,2.9701,10


In [16]:
dqn=DQN(4,2)
agent=Agent(dqn,cbs=[ArgMaxFeed,DiscreteEpsilonRandomSelect])
source=Source(cbs=[GymLoop('CartPole-v1',agent,steps_count=3,seed=0,
                           steps_delta=1),FirstLast])
dls=SourceDataBlock().dataloaders([source],n=1000,bs=1,num_workers=0)

learn=Learner(dls,agent,loss_func=MSELoss(),
              cbs=[ExperienceReplay(bs=32,max_sz=100000,warmup_sz=32),DQNTargetTrainer(n_steps=3)],
              metrics=[Reward,Epsilon,NEpisodes])

Could not do one pass in your dataloader, there is something wrong in it


In [17]:
slow=False
learn.fit(3 if slow else 47,lr=0.0001,wd=0)

KeyboardInterrupt: 

In [14]:
# hide
from fastcore.imports import in_colab

# Since colab still requires tornado<6, we don't want to import nbdev if we don't have to
if not in_colab():
    from nbdev.export import *
    from nbdev.export2html import *
    from nbdev.cli import make_readme
    make_readme()
    notebook2script()
    notebook2html()

converting /home/fastrl_user/fastrl/nbs/index.ipynb to README.md
Converted 00_core.ipynb.
Converted 00_nbdev_extension.ipynb.
Converted 03_callback.core.ipynb.
Converted 04_agent.ipynb.
Converted 05_data.test_async.ipynb.
Converted 05a_data.block.ipynb.
Converted 05b_data.block_simple.ipynb.
Converted 05c_data.gym.ipynb.
Converted 10a_agents.dqn.core.ipynb.
Converted 10b_agents.dqn.targets.ipynb.
Converted 10c_agents.dqn.double.ipynb.
Converted 10d_agents.dqn.dueling.ipynb.
Converted 10e_agents.dqn.categorical.ipynb.
Converted 11a_agents.policy_gradient.ppo.ipynb.
Converted 20_test_utils.ipynb.
Converted index.ipynb.
Converted nbdev_template.ipynb.
No notebooks were modified
